In [1]:
from iotbx.map_manager import map_manager as MapManager
from mmtbx.model import manager as ModelManager
from iotbx.data_manager import DataManager

In [2]:
import os
import pickle
from multiprocessing import Pool
from collections import Counter

In [3]:
errors = []
error_log_path = "data/errors.txt"

maps_and_models_path = "/net/cci/share/cryoem/maps_and_models/" # this directory is curated EM structures

entry_ids = [entry for entry in os.listdir(maps_and_models_path) if os.path.isdir(maps_and_models_path+entry)]


entries = [] # this is a list of group_args objects

# read the group_args objects in from the .pkl files in the maps_and_models directory
for entry_id in entry_ids:
    entry_path = maps_and_models_path+entry_id
    filenames = os.listdir(entry_path)
    extensions = [filename.split(".")[-1] for filename in filenames]
    extension_dict = Counter(extensions)
    if extension_dict["pkl"] == 1:
        pkl_path = entry_path+"/"+[filename for filename in filenames if filename.split(".")[-1] == "pkl"][0]
        with open(pkl_path,"rb") as fh:
            group_args = pickle.load(fh)
        group_args.add(key="entry",value=entry)
        group_args.add(key="entry_pdb",value=entry[:4])
        group_args.add(key="entry_emdb",value=entry[5:])
        entries.append(group_args)

    else:
        errors.append("FAILED: initial pickle read: "+entry_path+": .pkl occurrences != 1")

with open(error_log_path,"w") as fh:
    for error in errors:
        fh.write(error+"\n")
        

In [6]:
def extract_entry_composition(group_args):
    """
    1. takes an entry group_args
    2. reads it in, gets composition
    3. adds composition to group_args
    4. returns group_args
    """
    try:
        dm = DataManager()
        dm.process_model_file(group_args.model_file)
        group_args.add(key="composition",value=dm.get_model().composition())
    except:
        group_args.add(key="composition",value="None")
    return group_args

In [7]:
# Extract composition in parallel
p = Pool(21)
results = p.map(extract_entry_composition,entries)

Process PoolWorker-3:
Process PoolWorker-18:
Process PoolWorker-5:
Process PoolWorker-16:
Process PoolWorker-4:
Process PoolWorker-9:
Process PoolWorker-10:
Process PoolWorker-1:
Process PoolWorker-20:
Process PoolWorker-13:
Process PoolWorker-15:
Process PoolWorker-8:
Process PoolWorker-7:
Process PoolWorker-21:
Process PoolWorker-6:
Process PoolWorker-12:
Process PoolWorker-19:
Process PoolWorker-11:
Process PoolWorker-14:
Process PoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent 

  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker
  File "/net/cci-filer3/home/cschlick/Software/phenix/conda_base/lib/python2.7/multiprocessing/pool.py", line 102, in worker


KeyboardInterrupt: 

In [49]:
# dump list of entries (group_arg objects) to disk
with open("data/entry_composition.pkl","wb") as fh:
    pickle.dump(results,fh)

In [58]:
# read it back in
entry_composition_path = "data/entry_composition.pkl" #list of group_args objects
with open(entry_composition_path,"rb") as fh:
    entry_composition_list = pickle.load(fh)

In [59]:
# prune for entries without a composition (todo: why do some not have composition?)
entries_no_composition = []
entries_yes_composition = []
ligand_list = []
for entry in entry_composition_list:
    if entry.composition == "None":
        entries_no_composition.append(entry)
    else:
        entries_yes_composition.append(entry)

In [ ]:
print(len(entries_no_composition))

In [61]:
# write list of entries to disk
with open("data/entry_composition.pkl","wb") as fh:
    pickle.dump(entries_yes_composition,fh)